In [1]:
import os
os.chdir("../")

In [2]:
import glob
import pandas as pd
import time
import numpy as np
from scipy import stats

In [76]:
resultDir = ''
problem = 'setcover' # choices=['setcover', 'cauctions', 'facilities', 'indset']
problemSizes = ['small','medium','big']
models = ['gcnn:Lodi','ml-competitor:extratrees_gcnn_agg','ml-competitor:lambdamart_khalil','gcnn:GraphConv'] # choices: uniform5, depthK, depthK2, depthK3
# 'internal:relpscost' 'ml-competitor:lambdamart_khalil'
seeds = [0,1,2,3,4]

In [79]:

df = pd.read_csv(f'{resultDir}\\setcover_20220425-102653.csv')
df

,policy,seed,type,instance,nnodes,nlps,stime,gap,status,ndomchgs,ncutoffs,walltime,proctime
0,gcnn:GraphConv,0,small,data/instances/setcover/transfer_500r_1000c_0....,243,501,8.27,0.000000,optimal,0,0,9.151300,8.968750
1,gcnn:GraphConv,1,small,data/instances/setcover/transfer_500r_1000c_0....,215,469,7.15,0.000000,optimal,0,0,7.464725,7.406250
2,gcnn:GraphConv,2,small,data/instances/setcover/transfer_500r_1000c_0....,229,466,7.23,0.000000,optimal,0,0,7.736947,7.671875
3,gcnn:GraphConv,3,small,data/instances/setcover/transfer_500r_1000c_0....,209,435,6.79,0.000000,optimal,0,0,7.197353,7.187500
4,gcnn:GraphConv,4,small,data/instances/setcover/transfer_500r_1000c_0....,235,515,7.56,0.000000,optimal,0,0,7.983513,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,ml-competitor:lambdamart_khalil,0,big,data/instances/setcover/transfer_2000r_1000c_0...,328895,425848,7200.00,0.061589,timelimit,0,0,7235.726158,7220.187500
1496,ml-competitor:lambdamart_khalil,1,big,data/instances/setcover/transfer_2000r_1000c_0...,391880,448483,7200.00,0.034264,timelimit,0,0,7239.433329,7221.375000
1497,ml-competitor:lambdamart_khalil,2,big,data/instances/setcover/transfer_2000r_1000c_0...,336441,424226,7200.00,0.052507,timelimit,0,0,7239.463913,7221.812500
1498,ml-competitor:lambdamart_khalil,3,big,data/instances/setcover/transfer_2000r_1000c_0...,331816,432492,7200.00,0.052680,timelimit,0,0,7239.069954,7221.890625


In [80]:
df = pd.concat([df[df['type'] == probSize] for probSize in problemSizes])
df = df.astype({'nlps': float, 'nnodes' : float})
df_gcnns = df[df['policy'] != 'internal:relpscost'].copy()
df_gcnns

,policy,seed,type,instance,nnodes,nlps,stime,gap,status,ndomchgs,ncutoffs,walltime,proctime
0,gcnn:GraphConv,0,small,data/instances/setcover/transfer_500r_1000c_0....,243.0,501.0,8.27,0.000000,optimal,0,0,9.151300,8.968750
1,gcnn:GraphConv,1,small,data/instances/setcover/transfer_500r_1000c_0....,215.0,469.0,7.15,0.000000,optimal,0,0,7.464725,7.406250
2,gcnn:GraphConv,2,small,data/instances/setcover/transfer_500r_1000c_0....,229.0,466.0,7.23,0.000000,optimal,0,0,7.736947,7.671875
3,gcnn:GraphConv,3,small,data/instances/setcover/transfer_500r_1000c_0....,209.0,435.0,6.79,0.000000,optimal,0,0,7.197353,7.187500
4,gcnn:GraphConv,4,small,data/instances/setcover/transfer_500r_1000c_0....,235.0,515.0,7.56,0.000000,optimal,0,0,7.983513,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,ml-competitor:lambdamart_khalil,0,big,data/instances/setcover/transfer_2000r_1000c_0...,328895.0,425848.0,7200.00,0.061589,timelimit,0,0,7235.726158,7220.187500
1496,ml-competitor:lambdamart_khalil,1,big,data/instances/setcover/transfer_2000r_1000c_0...,391880.0,448483.0,7200.00,0.034264,timelimit,0,0,7239.433329,7221.375000
1497,ml-competitor:lambdamart_khalil,2,big,data/instances/setcover/transfer_2000r_1000c_0...,336441.0,424226.0,7200.00,0.052507,timelimit,0,0,7239.463913,7221.812500
1498,ml-competitor:lambdamart_khalil,3,big,data/instances/setcover/transfer_2000r_1000c_0...,331816.0,432492.0,7200.00,0.052680,timelimit,0,0,7239.069954,7221.890625


In [81]:
def gmean_1shifted(x):
    return stats.mstats.gmean(x + 1) - 1

# 1. Means

In [1]:
dfgcnns_gmean = df_gcnns.groupby(['type','policy'])[['nnodes', 'stime']].agg(gmean_1shifted)
dfgcnns_mean = df_gcnns.groupby(['type','policy'])[['nnodes', 'stime']].mean()

NameError: name 'df_gcnns' is not defined

In [83]:
def norm_by_uniform5(dfgcnns_grouped):
    df_list = []
    for probSize in dfgcnns_grouped.index.levels[0]:
        df_list.append(dfgcnns_grouped.loc[probSize] / dfgcnns_grouped.loc[(probSize, 'gcnn:Lodi')])
    return pd.concat(df_list, keys=dfgcnns_grouped.index.levels[0])

In [84]:
dfgcnns_gmean_norm = norm_by_uniform5(dfgcnns_gmean)
dfgcnns_mean_norm = norm_by_uniform5(dfgcnns_mean)
dfgcnns_gmean_norm

nnodes     stime
type   policy                                               
big    gcnn:GraphConv                     0.746607  0.788272
       gcnn:Lodi                          1.000000  1.000000
       ml-competitor:extratrees_gcnn_agg  0.868707  1.174534
       ml-competitor:lambdamart_khalil    1.422046  1.035300
medium gcnn:GraphConv                     0.893890  0.954294
       gcnn:Lodi                          1.000000  1.000000
       ml-competitor:extratrees_gcnn_agg  1.348838  1.532070
       ml-competitor:lambdamart_khalil    1.706780  0.988515
small  gcnn:GraphConv                     0.992095  0.982097
       gcnn:Lodi                          1.000000  1.000000
       ml-competitor:extratrees_gcnn_agg  1.445695  1.174168
       ml-competitor:lambdamart_khalil    1.418089  0.801096

# 2. Std variances (per instance)

In [85]:
dfgcnns_std_norm_perInstance = df_gcnns.groupby(['type','policy','instance']).std() / df_gcnns.groupby(['type','policy','instance']).mean()

In [86]:
dfgcnns_std_norm = dfgcnns_std_norm_perInstance.groupby(['type','policy'])[['nnodes','stime']].mean()
dfgcnns_std_norm

nnodes     stime
type   policy                                               
big    gcnn:GraphConv                     0.082077  0.066782
       gcnn:Lodi                          0.316227  0.294235
       ml-competitor:extratrees_gcnn_agg  0.137088  0.092596
       ml-competitor:lambdamart_khalil    0.202777  0.176806
medium gcnn:GraphConv                     0.090489  0.079115
       gcnn:Lodi                          0.165918  0.157053
       ml-competitor:extratrees_gcnn_agg  0.147979  0.125972
       ml-competitor:lambdamart_khalil    0.275905  0.227040
small  gcnn:GraphConv                     0.068193  0.043242
       gcnn:Lodi                          0.092767  0.052833
       ml-competitor:extratrees_gcnn_agg  0.201094  0.108712
       ml-competitor:lambdamart_khalil    0.178195  0.081100

# 做一下 T-Test

# 计算Wins

In [87]:
ttest_res = pd.DataFrame(index=dfgcnns_mean_norm.index, columns=pd.MultiIndex.from_product((['nnodes', 'stime'], ['t_stats', 'p_value'])))
for metric in ['nnodes', 'stime']:
    for probSize in dfgcnns_mean_norm.index.levels[0]: # medium, small
        mean2 = 1
        std2 = dfgcnns_std_norm.at[(probSize,'gcnn:Lodi'),metric]
        for policy in dfgcnns_mean_norm.index.levels[1]: # depthK, depthK2, uniform5
            mean1 = dfgcnns_mean_norm.at[(probSize,policy),metric]
            std1_norm = dfgcnns_std_norm.at[(probSize,policy),metric]
            std1 = std1_norm * mean1
            t_stats, p_value = stats.ttest_ind_from_stats(mean1, std1, 100, mean2, std2, 100)
            ttest_res.loc[(probSize,policy),(metric,)] = [t_stats, p_value]

In [88]:
ttest_res

nnodes                stime  \
                                            t_stats   p_value    t_stats   
type   policy                                                              
big    gcnn:GraphConv                      -4.37919  0.000019   -4.03942   
       gcnn:Lodi                                0.0       1.0        0.0   
       ml-competitor:extratrees_gcnn_agg  -6.134498       0.0   1.928275   
       ml-competitor:lambdamart_khalil     8.948755       0.0   0.104841   
medium gcnn:GraphConv                     -8.427626       0.0   -4.97998   
       gcnn:Lodi                                0.0       1.0        0.0   
       ml-competitor:extratrees_gcnn_agg  12.076707       0.0  22.496855   
       ml-competitor:lambdamart_khalil    13.900538       0.0   0.519024   
small  gcnn:GraphConv                      0.940628  0.348042  -0.360899   
       gcnn:Lodi                                0.0       1.0        0.0   
       ml-competitor:extratrees_gcnn_agg   19.32524       0.0  26.718776   
       ml-competitor:lambdamart_khalil    20.470308       0.0 -26.369081   

                                                    
                                           p_value  
type   policy                                       
big    gcnn:GraphConv                     0.000077  
       gcnn:Lodi                               1.0  
       ml-competitor:extratrees_gcnn_agg  0.055251  
       ml-competitor:lambdamart_khalil    0.916608  
medium gcnn:GraphConv                     0.000001  
       gcnn:Lodi                               1.0  
       ml-competitor:extratrees_gcnn_agg       0.0  
       ml-competitor:lambdamart_khalil    0.604323  
small  gcnn:GraphConv                     0.718559  
       gcnn:Lodi                               1.0  
       ml-competitor:extratrees_gcnn_agg       0.0  
       ml-competitor:lambdamart_khalil         0.0

In [89]:
winner_idx = df_gcnns.groupby(['type','instance','seed'])['stime'].agg(pd.Series.idxmin)
winner_idx

type   instance                                                          seed
big    data/instances/setcover/transfer_2000r_1000c_0.05d/instance_1.lp  0       400.0
                                                                         1       401.0
                                                                         2       402.0
                                                                         3       403.0
                                                                         4       404.0
                                                                                 ...  
small  data/instances/setcover/transfer_500r_1000c_0.05d/instance_9.lp   0       730.0
                                                                         1       731.0
                                                                         2       732.0
                                                                         3       733.0
                                                    

In [90]:
df_gcnns['wins'] = 0
df_gcnns.loc[winner_idx,'wins'] = 1

In [91]:
len(winner_idx)

300

In [92]:
dfgcnns_wins = df_gcnns.groupby(['type', 'policy'])['wins'].sum()
dfgcnns_wins

type    policy                           
big     gcnn:GraphConv                       67
        gcnn:Lodi                            20
        ml-competitor:extratrees_gcnn_agg     3
        ml-competitor:lambdamart_khalil      10
medium  gcnn:GraphConv                       34
        gcnn:Lodi                            21
        ml-competitor:extratrees_gcnn_agg     0
        ml-competitor:lambdamart_khalil      45
small   gcnn:GraphConv                        3
        gcnn:Lodi                             2
        ml-competitor:extratrees_gcnn_agg     0
        ml-competitor:lambdamart_khalil      95
Name: wins, dtype: int64

In [93]:
stats.ttest_ind_from_stats(0.967777, 0.085076*0.967777, 100, 1, 0.102216, 100)

Ttest_indResult(statistic=-2.455047672770633, pvalue=0.014949535302005438)

# 把Means、std 和 Wins合并起来

In [94]:
pd.concat([dfgcnns_gmean_norm, dfgcnns_std_norm, dfgcnns_wins], axis=1, keys=['gmean','std','wins'])

gmean                 std  \
                                            nnodes     stime    nnodes   
type   policy                                                            
big    gcnn:GraphConv                     0.746607  0.788272  0.082077   
       gcnn:Lodi                          1.000000  1.000000  0.316227   
       ml-competitor:extratrees_gcnn_agg  0.868707  1.174534  0.137088   
       ml-competitor:lambdamart_khalil    1.422046  1.035300  0.202777   
medium gcnn:GraphConv                     0.893890  0.954294  0.090489   
       gcnn:Lodi                          1.000000  1.000000  0.165918   
       ml-competitor:extratrees_gcnn_agg  1.348838  1.532070  0.147979   
       ml-competitor:lambdamart_khalil    1.706780  0.988515  0.275905   
small  gcnn:GraphConv                     0.992095  0.982097  0.068193   
       gcnn:Lodi                          1.000000  1.000000  0.092767   
       ml-competitor:extratrees_gcnn_agg  1.445695  1.174168  0.201094   
       ml-competitor:lambdamart_khalil    1.418089  0.801096  0.178195   

                                                   wins  
                                             stime wins  
type   policy                                            
big    gcnn:GraphConv                     0.066782   67  
       gcnn:Lodi                          0.294235   20  
       ml-competitor:extratrees_gcnn_agg  0.092596    3  
       ml-competitor:lambdamart_khalil    0.176806   10  
medium gcnn:GraphConv                     0.079115   34  
       gcnn:Lodi                          0.157053   21  
       ml-competitor:extratrees_gcnn_agg  0.125972    0  
       ml-competitor:lambdamart_khalil    0.227040   45  
small  gcnn:GraphConv                     0.043242    3  
       gcnn:Lodi                          0.052833    2  
       ml-competitor:extratrees_gcnn_agg  0.108712    0  
       ml-competitor:lambdamart_khalil    0.081100   95

In [95]:
# output_idx = pd.MultiIndex.from_product((dfgcnns_gmean_norm.index.get_level_values('type').unique(), sampling_Strategies), names=['type', 'sampling_strategy'])
output_idx = pd.MultiIndex.from_product((problemSizes, models), names=['type', 'policy'])
output_idx

MultiIndex([( 'small',                         'gcnn:Lodi'),
            ( 'small', 'ml-competitor:extratrees_gcnn_agg'),
            ( 'small',   'ml-competitor:lambdamart_khalil'),
            ( 'small',                    'gcnn:GraphConv'),
            ('medium',                         'gcnn:Lodi'),
            ('medium', 'ml-competitor:extratrees_gcnn_agg'),
            ('medium',   'ml-competitor:lambdamart_khalil'),
            ('medium',                    'gcnn:GraphConv'),
            (   'big',                         'gcnn:Lodi'),
            (   'big', 'ml-competitor:extratrees_gcnn_agg'),
            (   'big',   'ml-competitor:lambdamart_khalil'),
            (   'big',                    'gcnn:GraphConv')],
           names=['type', 'policy'])

In [96]:
output_df_mean = dfgcnns_gmean_norm.reindex(output_idx)
output_df_std = dfgcnns_std_norm.reindex(output_idx)
output_df_wins = dfgcnns_wins.reindex(output_idx)
output_df_ttest = ttest_res.reindex(output_idx)

In [97]:
out_df1 = pd.DataFrame(index=output_idx, columns=['nnodes', 'stime', 'wins', 't-stats (p-val)'])
out_df1

nnodes stime wins t-stats (p-val)
type   policy                                                             
small  gcnn:Lodi                            NaN   NaN  NaN             NaN
       ml-competitor:extratrees_gcnn_agg    NaN   NaN  NaN             NaN
       ml-competitor:lambdamart_khalil      NaN   NaN  NaN             NaN
       gcnn:GraphConv                       NaN   NaN  NaN             NaN
medium gcnn:Lodi                            NaN   NaN  NaN             NaN
       ml-competitor:extratrees_gcnn_agg    NaN   NaN  NaN             NaN
       ml-competitor:lambdamart_khalil      NaN   NaN  NaN             NaN
       gcnn:GraphConv                       NaN   NaN  NaN             NaN
big    gcnn:Lodi                            NaN   NaN  NaN             NaN
       ml-competitor:extratrees_gcnn_agg    NaN   NaN  NaN             NaN
       ml-competitor:lambdamart_khalil      NaN   NaN  NaN             NaN
       gcnn:GraphConv                       NaN   NaN  NaN             NaN

In [98]:
for col_name in ['nnodes', 'stime']:
    out_df1[col_name] = ["%.4fr ± %.2f" % (m, s*100) + "%" for (m, s) in zip(output_df_mean[col_name], output_df_std[col_name])]
out_df1['wins'] = output_df_wins

In [99]:
out_df1['t-stats (p-val)'] = ["%.2f(%.4f)" % (t, p) for (t, p) in zip(output_df_ttest[('stime','t_stats')], output_df_ttest[('stime','p_value')])]

In [100]:
out_df1

nnodes             stime  \
type   policy                                                                  
small  gcnn:Lodi                           1.0000r ± 9.28%   1.0000r ± 5.28%   
       ml-competitor:extratrees_gcnn_agg  1.4457r ± 20.11%  1.1742r ± 10.87%   
       ml-competitor:lambdamart_khalil    1.4181r ± 17.82%   0.8011r ± 8.11%   
       gcnn:GraphConv                      0.9921r ± 6.82%   0.9821r ± 4.32%   
medium gcnn:Lodi                          1.0000r ± 16.59%  1.0000r ± 15.71%   
       ml-competitor:extratrees_gcnn_agg  1.3488r ± 14.80%  1.5321r ± 12.60%   
       ml-competitor:lambdamart_khalil    1.7068r ± 27.59%  0.9885r ± 22.70%   
       gcnn:GraphConv                      0.8939r ± 9.05%   0.9543r ± 7.91%   
big    gcnn:Lodi                          1.0000r ± 31.62%  1.0000r ± 29.42%   
       ml-competitor:extratrees_gcnn_agg  0.8687r ± 13.71%   1.1745r ± 9.26%   
       ml-competitor:lambdamart_khalil    1.4220r ± 20.28%  1.0353r ± 17.68%   
       gcnn:GraphConv                      0.7466r ± 8.21%   0.7883r ± 6.68%   

                                          wins t-stats (p-val)  
type   policy                                                   
small  gcnn:Lodi                             2    0.00(1.0000)  
       ml-competitor:extratrees_gcnn_agg     0   26.72(0.0000)  
       ml-competitor:lambdamart_khalil      95  -26.37(0.0000)  
       gcnn:GraphConv                        3   -0.36(0.7186)  
medium gcnn:Lodi                            21    0.00(1.0000)  
       ml-competitor:extratrees_gcnn_agg     0   22.50(0.0000)  
       ml-competitor:lambdamart_khalil      45    0.52(0.6043)  
       gcnn:GraphConv                       34   -4.98(0.0000)  
big    gcnn:Lodi                            20    0.00(1.0000)  
       ml-competitor:extratrees_gcnn_agg     3    1.93(0.0553)  
       ml-competitor:lambdamart_khalil      10    0.10(0.9166)  
       gcnn:GraphConv                       67   -4.04(0.0001)